In [ ]:
from huggingface_hub import login

# 🔑 Autenticación en Hugging Face
login("")

# **Evaluación Qwen/Qwen2.5-3B-Instruct-AWQ - Intento 1**

In [ ]:
!pip install autoawq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00

# **Evaluación Qwen/Qwen2.5-3B-Instruct-AWQ en Español - Intento 1**

In [ ]:
# 1. Importar librerías
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 2. Cargar dataset desde Excel
file_path = "/content/BenchmarkSpanish.xlsx"
df = pd.read_excel(file_path)

# 3. Configurar modelo cuantizado (versión para Transformers)
model_id = "Qwen/Qwen2.5-3B-Instruct-AWQ"

# 4. Cargar tokenizer y modelo
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",  # Detecta automáticamente la GPU
    trust_remote_code=True  # Por si es necesario para la arquitectura
)

# 5. Parámetros de generación (equivalente al pipeline original)
generation_config = {
    "do_sample": True,
    "temperature": 0.01,
    "top_p": 0.9,
    "repetition_penalty": 1.1,
    "max_new_tokens": 512
}

# 6. Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosSpanishQwenQ1.xlsx", index=False)
print("✅ Archivo guardado: ResultadosSpanishQwenQ1.xlsx")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/839 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

🔹 Question 1: Los polinomios cuadráticos $ P (x) $ y $ Q (x) $ tienen coeficientes principales $ 2 $ y $ -2, $ respectivamente. Los gráficos de ambos polinomios pasan a través de los dos puntos $ (16,54) $ y $ (20,53). $ Encuentra $ P (0) + Q (0). $
🔸 Answer: To solve this problem, we need to determine the coefficients of the polynomials \(P(x)\) and \(Q(x)\), then find their values at \(x = 0\), and finally sum these values.

Given:
- The polynomial \(P(x)\) has a leading coefficient of 2.
- The polynomial \(Q(x)\) has a leading coefficient of -2.
- Both polynomials pass through the points \((16, 54)\) and \((20, 53)\).

Let's start by expressing the general form of the polynomials:

\[ P(x) = 2x^2 + ax + b \]
\[ Q(x) = -2x^2 + cx + d \]

Since both polynomials pass through the point \((16, 54)\):

For \(P(x)\):
\[ 2(16)^2 + 16a + b = 54 \]
\[ 2 \cdot 256 + 16a + b = 54 \]
\[ 512 + 16a + b = 54 \]
\[ 16a + b = 54 - 512 \]
\[ 16a + b = -458 \quad \text{(Equation 1)} \]

For \(Q(x)\):
\

# **Intento 2**

In [ ]:
#Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosSpanishQwenQ2.xlsx", index=False)
print("✅ Archivo guardado: ResultadosSpanishQwenQ2.xlsx")

🔹 Question 1: Los polinomios cuadráticos $ P (x) $ y $ Q (x) $ tienen coeficientes principales $ 2 $ y $ -2, $ respectivamente. Los gráficos de ambos polinomios pasan a través de los dos puntos $ (16,54) $ y $ (20,53). $ Encuentra $ P (0) + Q (0). $
🔸 Answer: To solve this problem, we need to determine the coefficients of the polynomials \(P(x)\) and \(Q(x)\), then find their values at \(x = 0\), and finally sum these values.

Given:
- The polynomial \(P(x)\) has a leading coefficient of 2.
- The polynomial \(Q(x)\) has a leading coefficient of -2.
- Both polynomials pass through the points \((16, 54)\) and \((20, 53)\).

Let's start by expressing the general form of the polynomials:

\[ P(x) = 2x^2 + ax + b \]
\[ Q(x) = -2x^2 + cx + d \]

Since both polynomials pass through the point \((16, 54)\):

For \(P(x)\):
\[ 2(16)^2 + 16a + b = 54 \]
\[ 2 \cdot 256 + 16a + b = 54 \]
\[ 512 + 16a + b = 54 \]
\[ 16a + b = 54 - 512 \]
\[ 16a + b = -458 \quad \text{(Equation 1)} \]

For \(Q(x)\):
\

# **Intento 3**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosSpanishQwenQ3.xlsx", index=False)
print("✅ Archivo guardado: ResultadosSpanishQwenQ3.xlsx")

🔹 Question 1: Los polinomios cuadráticos $ P (x) $ y $ Q (x) $ tienen coeficientes principales $ 2 $ y $ -2, $ respectivamente. Los gráficos de ambos polinomios pasan a través de los dos puntos $ (16,54) $ y $ (20,53). $ Encuentra $ P (0) + Q (0). $
🔸 Answer: To solve this problem, we need to determine the coefficients of the polynomials \(P(x)\) and \(Q(x)\), then find their values at \(x = 0\), and finally sum these values.

Given:
- The polynomial \(P(x)\) has a leading coefficient of 2.
- The polynomial \(Q(x)\) has a leading coefficient of -2.
- Both polynomials pass through the points \((16, 54)\) and \((20, 53)\).

Let's start by expressing the general form of the polynomials:

\[ P(x) = 2x^2 + ax + b \]
\[ Q(x) = -2x^2 + cx + d \]

Since both polynomials pass through the point \((16, 54)\):

For \(P(x)\):
\[ 2(16)^2 + 16a + b = 54 \]
\[ 2 \cdot 256 + 16a + b = 54 \]
\[ 512 + 16a + b = 54 \]
\[ 16a + b = 54 - 512 \]
\[ 16a + b = -458 \quad \text{(Equation 1)} \]

For \(Q(x)\):
\

# **Evaluación Inglés - Intento 1**

In [ ]:
#Cargar dataset desde Excel
file_path = "/content/BenchmarkEnglish.xlsx"
df = pd.read_excel(file_path)

# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosEnglishQwenQ1.xlsx", index=False)
print("✅ Archivo guardado: ResultadosEnglishQwenQ1.xlsx")


🔹 Question 1: Quadratic polynomials $P(x)$ and $Q(x)$ have leading coefficients $2$ and $-2,$ respectively. The graphs of both polynomials pass through the two points $(16,54)$ and $(20,53).$ Find $P(0) + Q(0).$
🔸 Answer: To solve for \( P(0) + Q(0) \), we start by expressing the quadratic polynomials in their general forms:
\[ P(x) = 2x^2 + bx + c \]
\[ Q(x) = -2x^2 + dx + e \]

Given that both polynomials pass through the points \((16, 54)\) and \((20, 53)\), we can substitute these points into the equations to form a system of linear equations.

First, substituting \((16, 54)\) into \(P(x)\):
\[
2(16)^2 + b(16) + c = 54
\]
\[
2 \cdot 256 + 16b + c = 54
\]
\[
512 + 16b + c = 54
\]
\[
16b + c = 54 - 512
\]
\[
16b + c = -458 \quad \text{(Equation 1)}
\]

Next, substituting \((16, 54)\) into \(Q(x)\):
\[
-2(16)^2 + d(16) + e = 54
\]
\[
-2 \cdot 256 + 16d + e = 54
\]
\[
-512 + 16d + e = 54
\]
\[
16d + e = 54 + 512
\]
\[
16d + e = 566 \quad \text{(Equation 2)}
\]

Now, substituting \((20,

# **Intento 2**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosEnglishQwenQ2.xlsx", index=False)
print("✅ Archivo guardado: ResultadosEnglishQwenQ2.xlsx")


🔹 Question 1: Quadratic polynomials $P(x)$ and $Q(x)$ have leading coefficients $2$ and $-2,$ respectively. The graphs of both polynomials pass through the two points $(16,54)$ and $(20,53).$ Find $P(0) + Q(0).$
🔸 Answer: To solve for \( P(0) + Q(0) \), we first need to determine the quadratic polynomials \( P(x) \) and \( Q(x) \).

Given that the leading coefficient of \( P(x) \) is 2, we can write:
\[ P(x) = 2x^2 + ax + b \]

Similarly, given that the leading coefficient of \( Q(x) \) is -2, we can write:
\[ Q(x) = -2x^2 + cx + d \]

We know that both polynomials pass through the points \((16, 54)\) and \((20, 53)\). Therefore, substituting these points into the equations will give us a system of linear equations.

First, substitute \((16, 54)\) into \( P(x) \):
\[ 2(16)^2 + a(16) + b = 54 \]
\[ 2 \cdot 256 + 16a + b = 54 \]
\[ 512 + 16a + b = 54 \]
\[ 16a + b = 54 - 512 \]
\[ 16a + b = -458 \quad \text{(Equation 1)} \]

Next, substitute \((16, 54)\) into \( Q(x) \):
\[ -2(16)^2 + c(

# **Intento 3**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosEnglishQwenQ3.xlsx", index=False)
print("✅ Archivo guardado: ResultadosEnglishQwenQ3.xlsx")


🔹 Question 1: Quadratic polynomials $P(x)$ and $Q(x)$ have leading coefficients $2$ and $-2,$ respectively. The graphs of both polynomials pass through the two points $(16,54)$ and $(20,53).$ Find $P(0) + Q(0).$
🔸 Answer: To solve for \( P(0) + Q(0) \), we start by expressing the quadratic polynomials in their general forms:
\[ P(x) = 2x^2 + bx + c \]
\[ Q(x) = -2x^2 + dx + e \]

Given that both polynomials pass through the points \((16, 54)\) and \((20, 53)\), we can substitute these points into the equations to form a system of linear equations.

First, substituting \((16, 54)\) into \(P(x)\):
\[
2(16)^2 + b(16) + c = 54
\]
\[
2 \cdot 256 + 16b + c = 54
\]
\[
512 + 16b + c = 54
\]
\[
16b + c = 54 - 512
\]
\[
16b + c = -458 \quad \text{(Equation 1)}
\]

Next, substituting \((16, 54)\) into \(Q(x)\):
\[
-2(16)^2 + d(16) + e = 54
\]
\[
-2 \cdot 256 + 16d + e = 54
\]
\[
-512 + 16d + e = 54
\]
\[
16d + e = 54 + 512
\]
\[
16d + e = 566 \quad \text{(Equation 2)}
\]

Now, substituting \((20,

# **Evaluación en Francés - Intento 1**

In [ ]:
#Cargar dataset desde Excel
file_path = "/content/BenchmarkFrench.xlsx"
df = pd.read_excel(file_path)

# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosFrenchQwenQ1.xlsx", index=False)
print("✅ Archivo guardado: ResultadosFrenchQwenQ1.xlsx")

🔹 Question 1: Les polynômes quadratiques $ p (x) $ et $ q (x) $ ont des coefficients de premier plan 2 $ et -2 $, $ respectivement. Les graphiques des deux polynômes passent par les deux points (16,54) $ et $ (20,53). $ Trouver $ p (0) + q (0). $
🔸 Answer: To solve this problem, we need to find the values of \(p(0)\) and \(q(0)\), where \(p(x)\) and \(q(x)\) are quadratic polynomials with specific coefficient constraints.

Given:
- The leading coefficient of \(p(x)\) is 2.
- The leading coefficient of \(q(x)\) is -2.
- Both polynomials pass through the points \((16, 54)\) and \((20, 53)\).

Let's start by expressing these polynomials in general form:

\[ p(x) = ax^2 + bx + c \]
\[ q(x) = dx^2 + ex + f \]

Since both have a leading coefficient of 2 for \(p(x)\) and -2 for \(q(x)\):

\[ p(x) = 2x^2 + bx + c \]
\[ q(x) = -2x^2 + ex + f \]

We know that both polynomials pass through the given points. Let's substitute these points into each polynomial equation to create a system of equation

# **Intento 2**

In [ ]:
#Cargar dataset desde Excel
file_path = "/content/BenchmarkFrench.xlsx"
df = pd.read_excel(file_path)

# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosFrenchQwenQ2.xlsx", index=False)
print("✅ Archivo guardado: ResultadosFrenchQwenQ2.xlsx")

🔹 Question 1: Les polynômes quadratiques $ p (x) $ et $ q (x) $ ont des coefficients de premier plan 2 $ et -2 $, $ respectivement. Les graphiques des deux polynômes passent par les deux points (16,54) $ et $ (20,53). $ Trouver $ p (0) + q (0). $
🔸 Answer: To solve this problem, we need to find the values of \(p(0)\) and \(q(0)\), where \(p(x)\) and \(q(x)\) are quadratic polynomials with specific coefficient constraints.

Given:
- The leading coefficient of \(p(x)\) is 2.
- The leading coefficient of \(q(x)\) is -2.
- Both polynomials pass through the points \((16, 54)\) and \((20, 53)\).

Let's start by expressing these polynomials in general form:

\[ p(x) = ax^2 + bx + c \]
\[ q(x) = dx^2 + ex + f \]

Since both have a leading coefficient of 2 for \(p(x)\) and -2 for \(q(x)\):

\[ p(x) = 2x^2 + bx + c \]
\[ q(x) = -2x^2 + ex + f \]

We know that both polynomials pass through the given points. Let's substitute these points into each polynomial equation to create a system of equation

# **Intento 3**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosFrenchQwenQ3.xlsx", index=False)
print("✅ Archivo guardado: ResultadosFrenchQwenQ3.xlsx")

🔹 Question 1: Les polynômes quadratiques $ p (x) $ et $ q (x) $ ont des coefficients de premier plan 2 $ et -2 $, $ respectivement. Les graphiques des deux polynômes passent par les deux points (16,54) $ et $ (20,53). $ Trouver $ p (0) + q (0). $
🔸 Answer: To solve this problem, we need to find the values of \(p(0)\) and \(q(0)\), where \(p(x)\) and \(q(x)\) are quadratic polynomials with specific coefficient constraints.

Given:
- The leading coefficient of \(p(x)\) is 2.
- The leading coefficient of \(q(x)\) is -2.
- Both polynomials pass through the points \((16, 54)\) and \((20, 53)\).

Let's start by expressing these polynomials in general form:

\[ p(x) = ax^2 + bx + c \]
\[ q(x) = dx^2 + ex + f \]

Since both have a leading coefficient of 2 for \(p(x)\) and -2 for \(q(x)\):

\[ p(x) = 2x^2 + bx + c \]
\[ q(x) = -2x^2 + ex + f \]

We know that both polynomials pass through the given points. Let's substitute these points into each polynomial equation to create a system of equation

# **Evaluación en Alemán - Intento 1**

In [ ]:
#Cargar dataset desde Excel
file_path = "/content/BenchmarkGerman.xlsx"
df = pd.read_excel(file_path)

# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosGermanQwenQ1.xlsx", index=False)
print("✅ Archivo guardado: ResultadosGermanQwenQ1.xlsx")

🔹 Question 1: Quadratische Polynome $P(x)$ und $Q(x)$ haben Leitkoeffizienten $2$ bzw. $-2,$. Die Graphen beider Polynome verlaufen durch die beiden Punkte $(16,54)$ und $(20,53).$ Berechnen Sie $P(0) + Q(0).$
🔸 Answer: To solve this problem, we need to find the values of \( P(0) \) and \( Q(0) \), where both polynomials have a leading coefficient of 2 for \( P(x) \) and -2 for \( Q(x) \). We are given that their graphs pass through the points (16, 54) and (20, 53).

First, let's express the general forms of \( P(x) \) and \( Q(x) \):
\[ P(x) = 2x^2 + ax + b \]
\[ Q(x) = -2x^2 + cx + d \]

Since both polynomials pass through the points (16, 54) and (20, 53), we can set up two systems of equations for each polynomial.

### For \( P(x) \):

Using the point (16, 54):
\[ 2(16)^2 + 16a + b = 54 \]
\[ 2(256) + 16a + b = 54 \]
\[ 512 + 16a + b = 54 \]
\[ 16a + b = 54 - 512 \]
\[ 16a + b = -458 \quad \text{(Equation 1)} \]

Using the point (20, 53):
\[ 2(20)^2 + 20a + b = 53 \]
\[ 2(400) + 20a

# **Intento 2**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosGermanQwenQ2.xlsx", index=False)
print("✅ Archivo guardado: ResultadosGermanQwenQ2.xlsx")

🔹 Question 1: Quadratische Polynome $P(x)$ und $Q(x)$ haben Leitkoeffizienten $2$ bzw. $-2,$. Die Graphen beider Polynome verlaufen durch die beiden Punkte $(16,54)$ und $(20,53).$ Berechnen Sie $P(0) + Q(0).$
🔸 Answer: To solve this problem, we need to find the values of \( P(0) \) and \( Q(0) \), where both polynomials have a leading coefficient of 2 for \( P(x) \) and -2 for \( Q(x) \). We are given that their graphs pass through the points (16, 54) and (20, 53).

First, let's express the polynomials in general form:
\[ P(x) = 2x^2 + ax + b \]
\[ Q(x) = -2x^2 + cx + d \]

Since both polynomials pass through the points (16, 54) and (20, 53), we can set up two systems of equations for each polynomial.

### For \( P(x) \):
Substitute (16, 54) into \( P(x) \):
\[ 2(16)^2 + 16a + b = 54 \]
\[ 2(256) + 16a + b = 54 \]
\[ 512 + 16a + b = 54 \]
\[ 16a + b = 54 - 512 \]
\[ 16a + b = -458 \quad \text{(Equation 1)} \]

Next, substitute (20, 53) into \( P(x) \):
\[ 2(20)^2 + 20a + b = 53 \]
\[ 

# **Intento 3**

In [ ]:
# Generación de respuestas
results = []

for idx, question in zip(df["id"], df["question"]):
    print(f"🔹 Question {idx}: {question}")

    prompt = f"""<|user|>

{question}
<|end|>
<|assistant|>"""

    # Tokenizar y mover a GPU si está disponible
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generar respuesta
    outputs = model.generate(
        **inputs,
        **generation_config
    )

    # Decodificar
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = decoded.replace(prompt, "").strip()

    print(f"🔸 Answer: {response}\n" + "-" * 80)

    results.append({
        "id": idx,
        "question": question,
        "answer": response
    })

# 7. Guardar resultados en Excel
df_results = pd.DataFrame(results)
df_results.to_excel("/content/ResultadosGermanQwenQ3.xlsx", index=False)
print("✅ Archivo guardado: ResultadosGermanQwenQ3.xlsx")

🔹 Question 1: Quadratische Polynome $P(x)$ und $Q(x)$ haben Leitkoeffizienten $2$ bzw. $-2,$. Die Graphen beider Polynome verlaufen durch die beiden Punkte $(16,54)$ und $(20,53).$ Berechnen Sie $P(0) + Q(0).$
🔸 Answer: To solve this problem, we need to find the values of \( P(0) \) and \( Q(0) \), where both polynomials have a leading coefficient of 2 for \( P(x) \) and -2 for \( Q(x) \). We are given that their graphs pass through the points (16, 54) and (20, 53).

First, let's express the polynomials in general form:
\[ P(x) = 2x^2 + ax + b \]
\[ Q(x) = -2x^2 + cx + d \]

Since both polynomials pass through the points (16, 54) and (20, 53), we can set up two systems of equations for each polynomial.

### For \( P(x) \):
Substitute (16, 54) into \( P(x) \):
\[ 2(16)^2 + 16a + b = 54 \]
\[ 2(256) + 16a + b = 54 \]
\[ 512 + 16a + b = 54 \]
\[ 16a + b = 54 - 512 \]
\[ 16a + b = -458 \quad \text{(Equation 1)} \]

Next, substitute (20, 53) into \( P(x) \):
\[ 2(20)^2 + 20a + b = 53 \]
\[ 